In [1]:
import json
from Gene import Gene
from Read import Read
from JunkReadRecovery import JunkReadRecovery

In [2]:
with open('./Simulation/sim_3_7.json') as f:
    data = json.load(f)

match_reward, mismatch_penalty, indel_penalty = 1, 1, 1
overlap_match_score, overlap_mismatch_score = 1, 1
threshold = 25
final_json = JunkReadRecovery(match_reward, mismatch_penalty, indel_penalty, overlap_match_score, overlap_mismatch_score, threshold, data, print_progress=True)

Aligning overlap reads


100%|██████████| 100/100 [00:49<00:00,  2.01it/s]



Aligning random reads


100%|██████████| 100/100 [00:49<00:00,  2.01it/s]


In [3]:
final_json

{'high_score_overlap': [{'final_score': 38,
   'read_seq': 'ACCTAGATCGCNACAGGTGTCCACTGGATCTAAATAACGTGGTCAATGGGTTCTATGAGCCTGCGTGGGCGACTN',
   'read_id': 0,
   'aligned_v_tail': 'ACCTAGATCGCNACAGGTGTCCACT',
   'aligned_read_head': 'ACCTAGATCGCNACAGGTGTCCACT',
   'v_gene_epi': ['TTHKSQKM'],
   'aligned_read_tail': 'TCCACTGGATCT-AA-A--TAACGTGGTCA-AT-GGGTTCT-ATGAGC-CTGC-GTGGG--CGACTN',
   'aligned_j_head': 'T-CA-AAGAT-TGAACACGCAACGT--TCACATCGGGCTCTGGTGAGCATTCCAANGGGACCG-CT-',
   'j_gene_epi': ['WTHSVFLA']},
  {'final_score': 57,
   'read_seq': 'ACACATCGTTGTGTTCGCTTCGGGCCATACTCATTAAAAGCGCTAGTCACGGCCNTAGTGGTCCGATGAGTCTGA',
   'read_id': 1,
   'aligned_v_tail': 'ACACATCGTTGTGTTCGCTTCGGGCCATACTCATTAAAAGCGCTA',
   'aligned_read_head': 'ACACATCGTTGTGTTCGCTTCGGGCCATACTCATTAAAAGCGCTA',
   'v_gene_epi': ['YLVAYNPW'],
   'aligned_read_tail': 'TCGTTGTGTTC-GCTTCGGGCCATACT-CATTA-AAAGCG-C-TAGTCA-CGGCCNTAG-T-GGTCCGATG-AGTC-TGA',
   'aligned_j_head': 'CCG--GCGGTCAAC-T--GACCAT-CTATATTAGTAA-CGCCATACTCATC-G--

In [4]:
from collections import defaultdict

def build_epitope_reads_dict(final_json):
    epitope_reads_dict = defaultdict(list)
    read_overlap = final_json['high_score_overlap']
    read_random = final_json['high_score_random']
    for read in read_overlap:
        for epitope in list(set(read['v_gene_epi'] + read['j_gene_epi'])):
            epitope_reads_dict[epitope].append(read['read_id'])
    for read in read_random:
        for epitope in list(set(read['v_gene_epi'] + read['j_gene_epi'])):
            epitope_reads_dict[epitope].append(-read['read_id'])
    return epitope_reads_dict

In [5]:
epitope_reads_dict = build_epitope_reads_dict(final_json)

In [6]:
epitope_reads_dict

defaultdict(list,
            {'WTHSVFLA': [0,
              2,
              3,
              4,
              5,
              7,
              8,
              9,
              10,
              12,
              13,
              17,
              19,
              20,
              21,
              22,
              24,
              26,
              27,
              29,
              33,
              37,
              38,
              39,
              41,
              43,
              44,
              48,
              49,
              51,
              53,
              54,
              56,
              60,
              62,
              65,
              67,
              68,
              70,
              74,
              78,
              79,
              80,
              81,
              83,
              86,
              88,
              89,
              92,
              93,
              94,
              96,
              97,
              0,
       

In [7]:
from typing import Dict, Set

def brute_force_max_coverage(epitope_reads_dict: Dict, k: int) -> Set:
    """
    Brute force algorithm to find the set of k epitopes that has the maximum coverage of reads.

    Parameters
    ----------
    epitope_reads_dict : Dict
        A dictionary with epitopes as keys and a list of reads as values.
    k : int
    """
    max_coverage = 0
    max_coverage_set = set()
    for read in epitope_reads_dict:
        for i in range(1, len(read)):
            for j in range(i, len(read) + 1):
                if len(set(read[i:j])) >= k:
                    if len(set(read[i:j])) > max_coverage:
                        max_coverage = len(set(read[i:j]))
                        max_coverage_set = set(read[i:j])
    return max_coverage_set

In [8]:
brute_force_max_coverage(epitope_reads_dict, 3)

{'A', 'F', 'H', 'L', 'S', 'T', 'V'}